**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

# https://stackoverflow.com/questions/60299967/how-to-get-allocated-gpu-spec-in-google-colab (How to get allocated GPU spec in Google Colab)

Sun Nov 22 19:32:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory
# https://www.atmarkit.co.jp/ait/articles/1605/30/news022.html (ln symbolic link)
# "!ln -s /content/gdrive/MyDrive/ /mydrive" also works well

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'				  W201_RDADA
 Linear_Algebra					  W203_Statistics
'My Drive'					  W205_Data_Engineering
 pdb2018trv4_us.csv				  W207_Applied_Machine_Learning
'Project 2 Ideas.gsheet'			  W251_Deep_Learning
 Sentences_50Agree.txt				  W266_NLP
 W200_Python					  W271_Time_Series
'W200_Satoshi_Final_PJ_2 - Line chart 1.gsheet'   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14449 (delta 14), reused 20 (delta 12), pack-reused 14421
Receiving objects: 100% (14449/14449), 13.18 MiB | 24.76 MiB/s, done.
Resolving deltas: 100% (9820/9820), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

# use "%cd" to move to repo
# https://qiita.com/nnn112358/items/746357a341ccf9ff6324

[Errno 2] No such file or directory: 'darknet'
/content
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
make: *** No targets specified and no makefile found.  Stop.


**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Gun" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj
# https://eng-entrance.com/linux-command-echo

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

# https://www.atmarkit.co.jp/ait/articles/1606/20/news024.html

--2020-11-22 19:34:02--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  9.96MB/s    in 16s     

2020-11-22 19:34:17 (9.84 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/001.jpg        
  inflating: data/obj/001.txt        
  inflating: data/obj/002.jpg        
  inflating: data/obj/002.txt        
  inflating: data/obj/003.jpg        
  inflating: data/obj/003.txt        
  inflating: data/obj/004.jpg        
  inflating: data/obj/004.txt        
  inflating: data/obj/005.jpg        
  inflating: data/obj/005.txt        
  inflating: data/obj/006.jpg        
  inflating: data/obj/006.txt        
  inflating: data/obj/007.jpg        
  inflating: data/obj/007.txt        
  inflating: data/obj/008.jpg        
  inflating: data/obj/008.txt        
  inflating: data/obj/009.jpg        
  inflating: data/obj/009.txt        
  inflating: data/obj/010.jpg        
  inflating: data/obj/010.txt        
  inflating: data/obj/011.jpg        
  inflating: data/obj/011.txt        
  inflating: data/obj/012.jpg        
  inflating: data/obj/012.txt        
  inflating: data/obj/013.jpg        
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.497596 0.495192 0.894231 0.884615

['0', '0.497596', '0.495192', '0.894231', '0.884615']
0 data/obj/038.txt
0 0.497596 0.495192 0.894231 0.884615
0 0.289663 0.469952 0.531250 0.319712

['0', '0.289663', '0.469952', '0.531250', '0.319712']
1 data/obj/048.txt
0 0.289663 0.469952 0.531250 0.319712
0 0.417067 0.491587 0.762019 0.920673

['0', '0.417067', '0.491587', '0.762019', '0.920673']
2 data/obj/103.txt
0 0.417067 0.491587 0.762019 0.920673
0 0.210337 0.348558 0.223558 0.591346

['0', '0.210337', '0.348558', '0.223558', '0.591346']
3 data/obj/058.txt
0 0.210337 0.348558 0.223558 0.591346
0 0.616538 0.464103 0.610000 0.561026

['0', '0.616538', '0.464103', '0.610000', '0.561026']
4 data/obj/004.txt
0 0.616538 0.464103 0.610000 0.561026
0 0.525625 0.483180 0.751250 0.715596

['0', '0.525625', '0.483180', '0.751250', '0.715596']
5 data/obj/130.txt
0 0.525625 0.483180 0.751250 0.715596
0 0.330000 0.285168 0.345000 0.285933

['0', '0.330000', '0.285168', '0.345000', '0.285933']
5 data/

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/038.jpg', 'data/obj/058.jpg', 'data/obj/113.jpg', 'data/obj/051.jpg', 'data/obj/048.jpg', 'data/obj/040.jpg', 'data/obj/079.jpg', 'data/obj/095.jpg', 'data/obj/015.jpg', 'data/obj/071.jpg', 'data/obj/065.jpg', 'data/obj/109.jpg', 'data/obj/022.jpg', 'data/obj/023.jpg', 'data/obj/021.jpg', 'data/obj/086.jpg', 'data/obj/063.jpg', 'data/obj/055.jpg', 'data/obj/098.jpg', 'data/obj/034.jpg', 'data/obj/102.jpg', 'data/obj/091.jpg', 'data/obj/057.jpg', 'data/obj/059.jpg', 'data/obj/127.jpg', 'data/obj/014.jpg', 'data/obj/039.jpg', 'data/obj/010.jpg', 'data/obj/078.jpg', 'data/obj/036.jpg', 'data/obj/080.jpg', 'data/obj/033.jpg', 'data/obj/029.jpg', 'data/obj/076.jpg', 'data/obj/073.jpg', 'data/obj/097.jpg', 'data/obj/060.jpg', 'data/obj/061.jpg', 'data/obj/120.jpg', 'data/obj/035.jpg', 'data/obj/075.jpg', 'data/obj/043.jpg', 'data/obj/123.jpg', 'data/obj/018.jpg', 'data/obj/089.jpg', 'data/obj/067.jpg', 'data/obj/106.jpg', 'data/obj/046.jpg', 'data/obj/004.jpg', 'data/obj/028.jpg',

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 252004, rewritten_bbox = 0.000000 % 

 3926: 0.047005, 0.047935 avg loss, 0.001000 rate, 10.083010 seconds, 251264 images, 0.332284 hours left
Loaded: 0.000039 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.932657, GIOU: 0.931826), Class: 0.999007, Obj: 0.818743, No Obj: 0.002452, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.058479, iou_loss = 0.055726, total_loss = 0.114205 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.912988, GIOU: 0.912987), Class: 0.997598, Obj: 0.521364, No Obj: 0.000107, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.210276, iou_loss = 0.046259, total_loss = 0.256535 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000004, iou_loss = 0.000000, total_loss = 0.000004 
 tot